In [1]:
filepath = "/usr/bin/ls"

In [2]:
!ls -l /usr/bin/ls

-rwxr-xr-x 1 root root 138208 Feb  7  2022 /usr/bin/ls


In [3]:
!file /usr/bin/ls

/usr/bin/ls: ELF 64-bit LSB pie executable, x86-64, version 1 (SYSV), dynamically linked, interpreter /lib64/ld-linux-x86-64.so.2, BuildID[sha1]=897f49cafa98c11d63e619e7e40352f855249c13, for GNU/Linux 3.2.0, stripped


In [4]:
!readelf -e -W /usr/bin/ls

ELF Header:
  Magic:   7f 45 4c 46 02 01 01 00 00 00 00 00 00 00 00 00 
  Class:                             ELF64
  Data:                              2's complement, little endian
  Version:                           1 (current)
  OS/ABI:                            UNIX - System V
  ABI Version:                       0
  Type:                              DYN (Position-Independent Executable file)
  Machine:                           Advanced Micro Devices X86-64
  Version:                           0x1
  Entry point address:               0x6ab0
  Start of program headers:          64 (bytes into file)
  Start of section headers:          136224 (bytes into file)
  Flags:                             0x0
  Size of this header:               64 (bytes)
  Size of program headers:           56 (bytes)
  Number of program headers:         13
  Size of section headers:           64 (bytes)
  Number of section headers:         31
  Section header string table index: 30

Section Headers:
  

In [5]:
!pip install pyelftools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.5/180.5 kB 3.1 MB/s eta 0:00:00


In [6]:
def dump(body):
    print("         0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F")
    ascii = ""
    i = 0
    l = 0
    print(f"{l:08X}:", end="")
    for byte in body:
        print(f"{byte:02X} ", end="")
        if byte >= 0x20 and byte <= 0x7e:
            ascii += chr(byte)
        else:
            ascii += '.'
        i += 1
        if i == 16:
            print(" ",ascii)
            ascii = ""
            i = 0
            l += 16
            print(f"{l:08X}:", end="")
    if i != 0:
        print("   "*(16-i),end="")
        print(" ",ascii)

def dump2(filepath):
    with open(filepath, "rb") as f:
        body = f.read()
        dump(body)


In [7]:
with open(filepath, "rb") as f:
    body = f.read()
    dump(body[:0x1000])


         0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F
00000000:7F 45 4C 46 02 01 01 00 00 00 00 00 00 00 00 00   .ELF............
00000010:03 00 3E 00 01 00 00 00 B0 6A 00 00 00 00 00 00   ..>......j......
00000020:40 00 00 00 00 00 00 00 20 14 02 00 00 00 00 00   @....... .......
00000030:00 00 00 00 40 00 38 00 0D 00 40 00 1F 00 1E 00   ....@.8...@.....
00000040:06 00 00 00 04 00 00 00 40 00 00 00 00 00 00 00   ........@.......
00000050:40 00 00 00 00 00 00 00 40 00 00 00 00 00 00 00   @.......@.......
00000060:D8 02 00 00 00 00 00 00 D8 02 00 00 00 00 00 00   ................
00000070:08 00 00 00 00 00 00 00 03 00 00 00 04 00 00 00   ................
00000080:18 03 00 00 00 00 00 00 18 03 00 00 00 00 00 00   ................
00000090:18 03 00 00 00 00 00 00 1C 00 00 00 00 00 00 00   ................
000000A0:1C 00 00 00 00 00 00 00 01 00 00 00 00 00 00 00   ................
000000B0:01 00 00 00 04 00 00 00 00 00 00 00 00 00 00 00   ................
000000C0:00 00 00 00 00 00 00 00

In [8]:
import elftools.elf.elffile

with open(filepath, "rb") as f:
    elf = elftools.elf.elffile.ELFFile(f)

    header = elf.header
    print(header)

Container({'e_ident': Container({'EI_MAG': [127, 69, 76, 70], 'EI_CLASS': 'ELFCLASS64', 'EI_DATA': 'ELFDATA2LSB', 'EI_VERSION': 'EV_CURRENT', 'EI_OSABI': 'ELFOSABI_SYSV', 'EI_ABIVERSION': 0}), 'e_type': 'ET_DYN', 'e_machine': 'EM_X86_64', 'e_version': 'EV_CURRENT', 'e_entry': 27312, 'e_phoff': 64, 'e_shoff': 136224, 'e_flags': 0, 'e_ehsize': 64, 'e_phentsize': 56, 'e_phnum': 13, 'e_shentsize': 64, 'e_shnum': 31, 'e_shstrndx': 30})


In [9]:
with open(filepath, "rb") as f:
    elf = elftools.elf.elffile.ELFFile(f)
    strtbl = elf.get_section(elf.header.e_shstrndx)
    dump(strtbl.data())


         0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F
00000000:00 2E 73 68 73 74 72 74 61 62 00 2E 69 6E 74 65   ..shstrtab..inte
00000010:72 70 00 2E 6E 6F 74 65 2E 67 6E 75 2E 70 72 6F   rp..note.gnu.pro
00000020:70 65 72 74 79 00 2E 6E 6F 74 65 2E 67 6E 75 2E   perty..note.gnu.
00000030:62 75 69 6C 64 2D 69 64 00 2E 6E 6F 74 65 2E 41   build-id..note.A
00000040:42 49 2D 74 61 67 00 2E 67 6E 75 2E 68 61 73 68   BI-tag..gnu.hash
00000050:00 2E 64 79 6E 73 79 6D 00 2E 64 79 6E 73 74 72   ..dynsym..dynstr
00000060:00 2E 67 6E 75 2E 76 65 72 73 69 6F 6E 00 2E 67   ..gnu.version..g
00000070:6E 75 2E 76 65 72 73 69 6F 6E 5F 72 00 2E 72 65   nu.version_r..re
00000080:6C 61 2E 64 79 6E 00 2E 72 65 6C 61 2E 70 6C 74   la.dyn..rela.plt
00000090:00 2E 69 6E 69 74 00 2E 70 6C 74 2E 67 6F 74 00   ..init..plt.got.
000000A0:2E 70 6C 74 2E 73 65 63 00 2E 74 65 78 74 00 2E   .plt.sec..text..
000000B0:66 69 6E 69 00 2E 72 6F 64 61 74 61 00 2E 65 68   fini..rodata..eh
000000C0:5F 66 72 61 6D 65 5F 68

In [10]:
with open(filepath, "rb") as f:
    elf = elftools.elf.elffile.ELFFile(f)

    strtbl = elf.get_section(elf.header.e_shstrndx)

    for s in elf.iter_sections():
        start_addr = s.header.sh_addr
        end_addr = s.header.sh_addr+s.header.sh_size
        name = strtbl.data()[s.header.sh_name:].split(b"\0", 1)[0].decode()

        print(f"{start_addr:08X}-{end_addr:08X}:{name}")


00000000-00000000:
00000318-00000334:.interp
00000338-00000368:.note.gnu.property
00000368-0000038C:.note.gnu.build-id
0000038C-000003AC:.note.ABI-tag
000003B0-000003FC:.gnu.hash
00000400-00000F88:.dynsym
00000F88-0000152E:.dynstr
0000152E-00001624:.gnu.version
00001628-000016E8:.gnu.version_r
000016E8-00002AC8:.rela.dyn
00002AC8-00003428:.rela.plt
00004000-00004025:.init
00004030-00004680:.plt
00004680-000046B0:.plt.got
000046B0-00004CF0:.plt.sec
00004CF0-00017131:.text
00017134-00017146:.fini
00018000-0001CDCC:.rodata
0001CDCC-0001D338:.eh_frame_hdr
0001D338-0001F458:.eh_frame
00021000-00021010:.ctors
00021010-00021020:.dtors
00021020-00021A98:.data.rel.ro
00021A98-00021C58:.dynamic
00021C58-00021FF8:.got
00022000-00022278:.data
00022280-00023540:.bss
00000000-00000049:.gnu_debugaltlink
00000000-00000034:.gnu_debuglink
00000000-00000125:.shstrtab


In [11]:
with open(filepath, "rb") as f:
    elf = elftools.elf.elffile.ELFFile(f)

    ep = elf.header.e_entry
    print(f"{ep:08X}")


00006AB0


In [12]:
with open(filepath, "rb") as f:
    elf = elftools.elf.elffile.ELFFile(f)
    ep = elf.header.e_entry

    ep_code = bytes(0)
    for s in elf.iter_sections():
        start_addr = s.header.sh_addr
        end_addr = s.header.sh_addr+s.header.sh_size
        if ep >= start_addr and ep < end_addr:
            ep_code = s.data()[ep-start_addr:]
            break
dump(ep_code[:0x100])


         0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F
00000000:F3 0F 1E FA 31 ED 49 89 D1 5E 48 89 E2 48 83 E4   ....1.I..^H..H..
00000010:F0 50 54 45 31 C0 31 C9 48 8D 3D 51 E2 FF FF FF   .PTE1.1.H.=Q....
00000020:15 C3 B4 01 00 F4 66 2E 0F 1F 84 00 00 00 00 00   ......f.........
00000030:48 8D 3D 91 B7 01 00 48 8D 05 8A B7 01 00 48 39   H.=....H......H9
00000040:F8 74 15 48 8B 05 A6 B4 01 00 48 85 C0 74 09 FF   .t.H......H..t..
00000050:E0 0F 1F 80 00 00 00 00 C3 0F 1F 80 00 00 00 00   ................
00000060:48 8D 3D 61 B7 01 00 48 8D 35 5A B7 01 00 48 29   H.=a...H.5Z...H)
00000070:FE 48 89 F0 48 C1 EE 3F 48 C1 F8 03 48 01 C6 48   .H..H..?H...H..H
00000080:D1 FE 74 14 48 8B 05 9D B4 01 00 48 85 C0 74 08   ..t.H......H..t.
00000090:FF E0 66 0F 1F 44 00 00 C3 0F 1F 80 00 00 00 00   ..f..D..........
000000A0:F3 0F 1E FA 80 3D 25 B7 01 00 00 75 7B 55 48 83   .....=%....u{UH.
000000B0:3D 82 B4 01 00 00 48 89 E5 41 54 53 74 0C 48 8B   =.....H..ATSt.H.
000000C0:3D 93 B4 01 00 E8 26 DB

In [13]:
!pip install capstone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.8 MB/s eta 0:00:00


In [14]:
from capstone import *
from capstone.x86 import *

with open(filepath, "rb") as f:
    elf = elftools.elf.elffile.ELFFile(f)

    # load disassembler
    if elf.header.e_machine == 'EM_X86_64':
        md = Cs(CS_ARCH_X86, CS_MODE_64)
    elif elf.header.e_machine == 'EM_386':
        md = Cs(CS_ARCH_X86, CS_MODE_32)
    else:
        assert False
    md.detail = True

    ep = elf.header.e_entry

    ep_code = bytes(0)
    for s in elf.iter_sections():
        start_addr = s.header.sh_addr
        end_addr = s.header.sh_addr+s.header.sh_size
        if ep >= start_addr and ep < end_addr:
            ep_code = s.data()[ep-start_addr:]
            break

for i in md.disasm(ep_code, ep,30):
    print(f"{i.address:08X}: {i.mnemonic: <8} {i.op_str}")


00006AB0: endbr64  
00006AB4: xor      ebp, ebp
00006AB6: mov      r9, rdx
00006AB9: pop      rsi
00006ABA: mov      rdx, rsp
00006ABD: and      rsp, 0xfffffffffffffff0
00006AC1: push     rax
00006AC2: push     rsp
00006AC3: xor      r8d, r8d
00006AC6: xor      ecx, ecx
00006AC8: lea      rdi, [rip - 0x1daf]
00006ACF: call     qword ptr [rip + 0x1b4c3]
00006AD5: hlt      
00006AD6: nop      word ptr cs:[rax + rax]
00006AE0: lea      rdi, [rip + 0x1b791]
00006AE7: lea      rax, [rip + 0x1b78a]
00006AEE: cmp      rax, rdi
00006AF1: je       0x6b08
00006AF3: mov      rax, qword ptr [rip + 0x1b4a6]
00006AFA: test     rax, rax
00006AFD: je       0x6b08
00006AFF: jmp      rax
00006B01: nop      dword ptr [rax]
00006B08: ret      
00006B09: nop      dword ptr [rax]
00006B10: lea      rdi, [rip + 0x1b761]
00006B17: lea      rsi, [rip + 0x1b75a]
00006B1E: sub      rsi, rdi
00006B21: mov      rax, rsi
00006B24: shr      rsi, 0x3f


In [15]:
for i in md.disasm(ep_code, ep,50):
    mc = ""
    for j in i.bytes:
        mc += f"{j:02X} "
    print(f"{i.address:08X}: {mc: <32}{i.id: <5}{i.mnemonic: <8} {i.op_str}")


00006AB0: F3 0F 1E FA                     165  endbr64  
00006AB4: 31 ED                           1503 xor      ebp, ebp
00006AB6: 49 89 D1                        460  mov      r9, rdx
00006AB9: 5E                              585  pop      rsi
00006ABA: 48 89 E2                        460  mov      rdx, rsp
00006ABD: 48 83 E4 F0                     24   and      rsp, 0xfffffffffffffff0
00006AC1: 50                              609  push     rax
00006AC2: 54                              609  push     rsp
00006AC3: 45 31 C0                        1503 xor      r8d, r8d
00006AC6: 31 C9                           1503 xor      ecx, ecx
00006AC8: 48 8D 3D 51 E2 FF FF            332  lea      rdi, [rip - 0x1daf]
00006ACF: FF 15 C3 B4 01 00               62   call     qword ptr [rip + 0x1b4c3]
00006AD5: F4                              223  hlt      
00006AD6: 66 2E 0F 1F 84 00 00 00 00 00   510  nop      word ptr cs:[rax + rax]
00006AE0: 48 8D 3D 91 B7 01 00            332  lea      rdi, [ri